<a href="https://colab.research.google.com/github/vkjadon/tf/blob/main/tf101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.2


##Load a dataset
Load and prepare the MNIST dataset. Convert the sample data from integers to floating-point numbers:

In [15]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

##Build a machine learning model
Build a tf.keras.Sequential model by stacking layers.

In [19]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [20]:
predictions = model(x_train[:1]).numpy()
#predictions = model(x_train[:1])
predictions
#predictions.dtype

array([[-0.5496927 , -0.29874387,  0.1736077 , -0.15129805,  0.07749262,
        -0.04680413,  0.7914394 ,  0.65793055,  0.694895  ,  0.3242821 ]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class:

In [21]:
tf.nn.softmax(predictions).numpy()

array([[0.04464577, 0.05738072, 0.09202498, 0.06649683, 0.08359176,
        0.07382137, 0.17069729, 0.14936349, 0.15498795, 0.1069899 ]],
      dtype=float32)

Note: It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using losses.SparseCategoricalCrossentropy, which takes a vector of logits and a True index and returns a scalar loss for each example.

In [12]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [13]:
loss_fn(y_train[:1], predictions).numpy()

2.1462982

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.

In [22]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

##Train and evaluate your model
Use the Model.fit method to adjust your model parameters and minimize the loss:

In [23]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2921 - accuracy: 0.9147
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1440 - accuracy: 0.9574
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1091 - accuracy: 0.9680
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0887 - accuracy: 0.9725
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0790 - accuracy: 0.9748


The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [24]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0769 - accuracy: 0.9765 - 607ms/epoch - 2ms/step


[0.07686926424503326, 0.9764999747276306]